# Semantic Kernel na may Integrasyon sa OpenBnB MCP Server

Ipinapakita ng notebook na ito kung paano gamitin ang Semantic Kernel kasama ang aktwal na OpenBnB MCP server upang maghanap ng mga tunay na Airbnb na tirahan gamit ang MCPStdioPlugin. Para sa LLM Access, gumagamit ito ng Azure AI Foundry. Upang i-setup ang iyong mga environment variable, maaari mong sundan ang [Setup Lesson](/00-course-setup/README.md)


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## Paglikha ng MCP Plugin Connection

Ikokonekta natin sa [OpenBnB MCP server](https://github.com/openbnb-org/mcp-server-airbnb) gamit ang MCPStdioPlugin. Ang server na ito ay nagbibigay ng functionality para sa paghahanap sa Airbnb sa pamamagitan ng @openbnb/mcp-server-airbnb package.


## Paglikha ng Client

Sa halimbawang ito, gagamitin natin ang Azure AI Foundry para sa ating LLM access. Siguraduhing maayos ang pagkaka-set up ng iyong mga environment variables.


## Konfigurasyon ng Kapaligiran

I-configure ang mga setting ng Azure OpenAI. Siguraduhing naitakda mo ang mga sumusunod na environment variables:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## Pag-unawa sa OpenBnB MCP Integration

Ang notebook na ito ay kumokonekta sa **tunay na OpenBnB MCP server** na nagbibigay ng aktwal na functionality sa paghahanap ng Airbnb.

### Paano ito gumagana:

1. **MCPStdioPlugin**: Gumagamit ng standard input/output na komunikasyon sa MCP server  
2. **Tunay na NPM Package**: Dina-download at pinapatakbo ang `@openbnb/mcp-server-airbnb` gamit ang npx  
3. **Live Data**: Nagbabalik ng aktwal na data ng mga property mula sa Airbnb APIs  
4. **Function Discovery**: Awtomatikong natutuklasan ng agent ang mga available na function mula sa MCP server  

### Mga Available na Function:

Karaniwang nagbibigay ang OpenBnB MCP server ng mga sumusunod:  
- **search_listings** - Maghanap ng mga property sa Airbnb batay sa lokasyon at mga pamantayan  
- **get_listing_details** - Kumuha ng detalyadong impormasyon tungkol sa mga partikular na property  
- **check_availability** - Tingnan ang availability para sa mga partikular na petsa  
- **get_reviews** - Kumuha ng mga review para sa mga property  
- **get_host_info** - Kumuha ng impormasyon tungkol sa mga host ng property  

### Mga Kinakailangan:

- **Node.js** na naka-install sa iyong sistema  
- **Internet connection** para ma-download ang MCP server package  
- **NPX** na magagamit (kasama na sa Node.js)  

### Pagsubok sa Koneksyon:

Maaari mong subukan ang MCP server nang manu-mano sa pamamagitan ng pagpapatakbo ng:  
```bash
npx -y @openbnb/mcp-server-airbnb
```

Ida-download at sisimulan nito ang OpenBnB MCP server, na kokonekta naman ang Semantic Kernel para sa aktwal na data mula sa Airbnb.  


## Pagpapatakbo ng Agent gamit ang OpenBnB MCP Server

Ngayon, patatakbuhin natin ang AI Agent na kumokonekta sa OpenBnB MCP server upang maghanap ng totoong Airbnb na tirahan sa Stockholm para sa 2 matatanda at 1 bata. Malaya kang baguhin ang `user_inputs` list upang i-modify ang mga pamantayan sa paghahanap.


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# Buod
Binabati kita! Matagumpay mong nabuo ang isang AI agent na may kakayahang mag-integrate sa aktwal na paghahanap ng tirahan gamit ang Model Context Protocol (MCP):

## Mga Teknolohiyang Ginamit:
- Semantic Kernel - Para sa pagbuo ng matatalinong ahente gamit ang Azure OpenAI
- Azure AI Foundry - Para sa LLM capabilities at chat completion
- MCP (Model Context Protocol) - Para sa standardized tool integration
- OpenBnB MCP Server - Para sa tunay na functionality ng Airbnb search
- Node.js/NPX - Para sa pagpapatakbo ng external MCP server

## Mga Natutunan Mo:
- MCP Integration: Pagkonekta ng Semantic Kernel agents sa external MCP servers
- Real-time Data Access: Paghahanap ng aktwal na mga ari-arian ng Airbnb sa pamamagitan ng live APIs
- Protocol Communication: Paggamit ng stdio communication sa pagitan ng agent at MCP server
- Function Discovery: Awtomatikong pagtuklas ng mga magagamit na function mula sa MCP servers
- Streaming Responses: Pagkuha at pag-log ng mga tawag sa function nang real-time
- HTML Rendering: Pag-format ng mga tugon ng agent gamit ang styled tables at interactive displays

## Mga Susunod na Hakbang:
- Mag-integrate ng karagdagang MCP servers (panahon, flights, restaurants)
- Bumuo ng multi-agent system na pinagsasama ang MCP at A2A protocols
- Gumawa ng custom MCP servers para sa sarili mong mga data sources
- Magpatupad ng persistent conversation memory sa bawat session
- I-deploy ang agent sa Azure Functions na may MCP server orchestration
- Magdagdag ng user authentication at booking capabilities



---

**Paunawa**:  
Ang dokumentong ito ay isinalin gamit ang AI translation service na [Co-op Translator](https://github.com/Azure/co-op-translator). Bagama't sinisikap naming maging tumpak, pakitandaan na ang mga awtomatikong pagsasalin ay maaaring maglaman ng mga pagkakamali o hindi pagkakatugma. Ang orihinal na dokumento sa orihinal nitong wika ang dapat ituring na opisyal na sanggunian. Para sa mahalagang impormasyon, inirerekomenda ang propesyonal na pagsasalin ng tao. Hindi kami mananagot sa anumang hindi pagkakaunawaan o maling interpretasyon na maaaring magmula sa paggamit ng pagsasaling ito.
